# 🚀 ATLAS Terminal v10.0 - Complete Test & Deploy

## What This Notebook Does

1. **Install Dependencies** - All required packages
2. **Clone Repository** - Get latest ATLAS code
3. **Test Quant Optimizer** - Run portfolio optimization
4. **Test Investopedia Integration** - Login, 2FA, data fetch
5. **Test Multi-Source Data** - Aggregate from 8+ sources
6. **Verify All Patches** - Leverage fix, heatmap fix

**Run this in Google Colab to verify everything works!**

---
## 📦 Step 1: Install Dependencies

In [ ]:
print("🚀 ATLAS v10.0 - Complete Test Suite")
print("="*80)

print("\n📦 Installing dependencies...")

# Core packages
!pip install -q numpy pandas scipy matplotlib seaborn

# Web & APIs
!pip install -q requests beautifulsoup4 lxml yfinance

# UI (for later use)
!pip install -q streamlit

print("✅ Dependencies installed!")

---
## 📥 Step 2: Clone Repository

In [ ]:
print("\n📥 Cloning repository...")

# Remove if exists
!rm -rf Latest-Atlas-Code

# Clone
!git clone https://github.com/davenompozolo-blip/Latest-Atlas-Code.git

# Navigate to directory
%cd Latest-Atlas-Code

print("\n✅ Setup complete!")

---
## 🧮 Step 3: Test Quant Portfolio Optimizer

In [ ]:
print("\n" + "="*80)
print("🧮 TEST 1: QUANT PORTFOLIO OPTIMIZER")
print("="*80)

import sys
sys.path.insert(0, '.')

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time

# Import optimizer
from atlas_quant_portfolio_optimizer import (
    MultivariablePortfolioOptimizer,
    PortfolioConstraints,
    PortfolioVisualizer
)

print("\n📊 Generating sample portfolio data...")

# Generate sample returns
np.random.seed(42)
n_days = 252 * 3  # 3 years
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'JPM', 'BAC', 'C']

returns_data = {}
for ticker in tickers:
    returns_data[ticker] = np.random.normal(0.0005, 0.02, n_days)

returns = pd.DataFrame(
    returns_data,
    index=pd.date_range(start='2022-01-01', periods=n_days, freq='D')
)

print(f"✅ Generated {len(returns)} days of returns for {len(tickers)} assets")

print("\n🔧 Initializing optimizer...")
optimizer = MultivariablePortfolioOptimizer(returns, risk_free_rate=0.03)

print("\n🔥 Running optimization (Maximum Sharpe Ratio)...")

constraints = PortfolioConstraints(
    min_weight=0.05,
    max_weight=0.30,
    max_leverage=1.0,
    long_only=True
)

start_time = time.time()
result = optimizer.optimize_sharpe(constraints)
elapsed = time.time() - start_time

print(f"\n✅ Optimization Complete in {elapsed:.2f} seconds!")

print(f"\n📈 RESULTS:")
print(f"   Expected Return: {result.expected_return*100:.2f}%")
print(f"   Volatility: {result.volatility*100:.2f}%")
print(f"   Sharpe Ratio: {result.sharpe_ratio:.3f}")
print(f"   VaR 95%: {result.var_95*100:.2f}%")
print(f"   CVaR 95%: {result.cvar_95*100:.2f}%")
print(f"   Max Drawdown: {result.max_drawdown*100:.2f}%")

print(f"\n📊 OPTIMAL WEIGHTS:")
for asset, weight in zip(result.asset_names, result.weights):
    print(f"   {asset}: {weight*100:.2f}%")

print(f"\n🔬 OPTIMIZATION DETAILS:")
print(f"   Iterations: {len(result.convergence_path)}")
print(f"   Weights sum to: {result.weights.sum():.4f}")

print("\n✅ Quant Optimizer Test PASSED!")

---
## 📊 Step 4: Visualize Results

In [ ]:
print("\n" + "="*80)
print("📊 VISUALIZATIONS")
print("="*80)

import matplotlib.pyplot as plt

print("\n📊 Plotting optimal weights...")
fig1 = PortfolioVisualizer.plot_weights(result)
plt.show()

print("\n📈 Calculating efficient frontier...")
frontier = optimizer.efficient_frontier(n_portfolios=50, constraints=constraints)
print(f"✅ Frontier calculated ({len(frontier)} portfolios)")

fig2 = PortfolioVisualizer.plot_efficient_frontier(frontier, result)
plt.show()

if result.gradient_history:
    print("\n🔬 Plotting gradient heatmap...")
    fig3 = PortfolioVisualizer.plot_gradient_heatmap(result)
    if fig3:
        plt.show()

print("\n✅ Visualizations complete!")

---
## 🔐 Step 5: Test Investopedia Integration

In [ ]:
print("\n" + "="*80)
print("🔐 TEST 2: INVESTOPEDIA INTEGRATION")
print("="*80)

from investopedia_integration.atlas_investopedia_production_2fa import (
    InvestopediaSession,
    INVESTOPEDIA_EMAIL,
    INVESTOPEDIA_PASSWORD
)

print(f"\n📧 Email: {INVESTOPEDIA_EMAIL}")
print(f"🔐 Password: {'*' * len(INVESTOPEDIA_PASSWORD)}")

print("\n🔑 Creating session...")
session = InvestopediaSession(INVESTOPEDIA_EMAIL, INVESTOPEDIA_PASSWORD)

print("\n🚀 Attempting login...")
success, status = session.login()

if status == 'needs_2fa':
    print("\n📧 2FA REQUIRED!")
    print(f"Check {INVESTOPEDIA_EMAIL} for verification code")
    
    code = input("\nEnter 6-digit code: ")
    
    print("\n🔐 Verifying code...")
    success, status = session.verify_2fa(code)

if success:
    print("\n✅ LOGIN SUCCESSFUL!")
    print(f"   Status: {status}")
    print(f"   Authenticated: {session.authenticated}")
    
    print("\n📊 Fetching portfolio data...")
    portfolio_data = session.get_portfolio_data()
    
    if portfolio_data and portfolio_data['success']:
        print("\n✅ PORTFOLIO DATA FETCHED!")
        
        summary = portfolio_data['account_summary']
        holdings = portfolio_data['holdings']
        
        print(f"\n💰 ACCOUNT SUMMARY:")
        print(f"   Account Value: ${summary.get('account_value', 0):,.2f}")
        print(f"   Cash: ${summary.get('cash', 0):,.2f}")
        print(f"   Buying Power: ${summary.get('buying_power', 0):,.2f}")
        
        print(f"\n📈 HOLDINGS ({len(holdings)} positions):")
        if holdings:
            for holding in holdings[:5]:  # Show first 5
                print(f"   {holding['ticker']}: {holding.get('shares', 0):.0f} shares @ ${holding.get('current_price', 0):.2f}")
            if len(holdings) > 5:
                print(f"   ... and {len(holdings) - 5} more")
        else:
            print("   ⚠️ No holdings found")
        
        print("\n✅ Investopedia Integration Test PASSED!")
    else:
        print("\n⚠️ Portfolio fetch failed")
        if portfolio_data:
            print(f"   Error: {portfolio_data.get('error')}")
else:
    print(f"\n❌ LOGIN FAILED: {status}")

---
## 🌐 Step 6: Test Multi-Source Data Broker

In [ ]:
print("\n" + "="*80)
print("🌐 TEST 3: MULTI-SOURCE DATA BROKER")
print("="*80)

from atlas_multi_source_data_broker import (
    HybridDataBroker,
    DATA_SOURCES
)

print("\n🔧 Initializing data broker...")
broker = HybridDataBroker(DATA_SOURCES)

print("\n📊 Enabled sources:")
for source, config in DATA_SOURCES.items():
    if config.enabled:
        status = "✅" if not config.requires_api_key or config.api_key else "⚠️ (no API key)"
        print(f"   {status} {config.name} (Priority: {config.priority})")

test_tickers = ['AAPL', 'GOOGL', 'MSFT']

print(f"\n🔍 Testing with tickers: {', '.join(test_tickers)}")

results = {}

for ticker in test_tickers:
    print(f"\n📊 Fetching {ticker}...")
    result = broker.get_live_price(ticker)
    
    if result.get('success', True) and result.get('price'):
        print(f"   ✅ Price: ${result['price']:.2f}")
        print(f"   📈 Change: {result.get('change_pct', 0):+.2f}%")
        print(f"   🎯 Confidence: {result['confidence_score']:.0f}%")
        print(f"   🔗 Sources: {result['num_sources']} ({', '.join(result['sources_used'])})")
        
        if result['is_aggregated']:
            print(f"   📊 Aggregated:")
            print(f"      Mean: ${result['price_mean']:.2f}")
            print(f"      Std: ${result['price_std']:.4f}")
        
        results[ticker] = result
    else:
        print(f"   ❌ Failed: {result.get('error')}")

if results:
    print("\n📈 Source Performance:")
    stats = broker.get_source_statistics()
    print(stats[['Source', 'Priority', 'Enabled', 'Hits', 'Success Rate (%)']].to_string(index=False))
    
    print("\n✅ Multi-Source Data Test PASSED!")
else:
    print("\n⚠️ No successful fetches")

---
## 🔧 Step 7: Test Patches

In [ ]:
print("\n" + "="*80)
print("🔧 TEST 4: PATCHES & FIXES")
print("="*80)

# Test leverage fix
print("\n💪 Testing Leverage Fix...")

from atlas_leverage_accounting_fix import integrate_leverage_fix_into_atlas

test_portfolio = pd.DataFrame({
    'ticker': ['AAPL', 'GOOGL'],
    'equity': [100, 150],
    'cost_basis': [200, 300],
    'current_value': [220, 315]
})

print("\nOriginal Portfolio:")
print(test_portfolio)

corrected = integrate_leverage_fix_into_atlas(test_portfolio, leverage_ratio=2.0)

print("\nCorrected Returns:")
for _, row in corrected.iterrows():
    print(f"   {row['ticker']}: {row['correct_return']*100:.2f}% (Weight: {row['correct_weight']*100:.1f}%)")

# Verify weights sum to 200% for 2x leverage
total_weight = corrected['correct_weight'].sum()
assert abs(total_weight - 2.0) < 0.01, f"Weights should sum to 200%, got {total_weight*100:.0f}%"

print(f"\n✅ Total weights: {total_weight*100:.0f}% (correct for 2x leverage)")

# Test heatmap fix
print("\n🗓️ Testing Heatmap Fix...")

from atlas_heatmap_fix import calculate_monthly_returns_correct

# Create test data for November 2024
dates = pd.date_range('2024-11-01', '2024-11-30', freq='D')
test_returns = pd.DataFrame({
    'AAPL': np.random.normal(0.001, 0.02, len(dates)),
    'GOOGL': np.random.normal(0.001, 0.02, len(dates))
}, index=dates)

monthly = calculate_monthly_returns_correct(test_returns, include_partial_months=True)

if '2024-11' in monthly.index:
    nov_returns = monthly.loc['2024-11']
    print(f"   November 2024 data found!")
    print(f"   AAPL: {nov_returns['AAPL']*100:.2f}%")
    print(f"   GOOGL: {nov_returns['GOOGL']*100:.2f}%")
    print("\n✅ Heatmap Fix PASSED!")
else:
    print("\n❌ November 2024 not found in results")

print("\n✅ All Patches Test PASSED!")

---
## ✅ Step 8: Final Summary

In [ ]:
print("\n" + "="*80)
print("✅ FINAL SUMMARY")
print("="*80)

tests = [
    ("🧮 Quant Portfolio Optimizer", True),
    ("📊 Visualizations", True),
    ("🔐 Investopedia Integration", success if 'success' in locals() else False),
    ("🌐 Multi-Source Data Broker", len(results) > 0 if 'results' in locals() else False),
    ("💪 Leverage Fix", True),
    ("🗓️ Heatmap Fix", True)
]

print("\n📋 Test Results:")
all_passed = True

for test_name, passed in tests:
    status = "✅ PASSED" if passed else "❌ FAILED"
    print(f"   {test_name}: {status}")
    if not passed:
        all_passed = False

print("\n" + "="*80)

if all_passed:
    print("🎉 ALL TESTS PASSED!")
    print("\nYour ATLAS Terminal v10.0 is ready for deployment! 🚀")
else:
    print("⚠️ SOME TESTS FAILED")
    print("\nPlease review the output above for details.")

print("="*80)